https://dacon.io/competitions/official/235622/codeshare/1591

신(新) 코로나 지역별 위험지표 제작 및 안전한 길찾기 플랫폼

국민의 안전한 이동을 위해 전국 **코로나 위험도**를 의미하는 위험지표를 나타내고 위험지표 중 큰 비중을 차지하는 **유동인구 데이터**를 활용한 안전경로 안내서비스

#### 1. 코로나 데이터 전처리 및 EDA 분석

In [ ]:
!pip install folium
!pip install jupyter_contrib_nbextensions && jupyter contrib nbextension install --user
%config InlineBackend.figure_format = 'retina'
!apt -qq -y install fonts-nanum
!pip install haversine
!pip install plotly_express
!pip install geopandas
!pip install mapboxgl
!pip install mapclassify

In [ ]:
import mapboxgl
import folium
import matplotlib.font_manager as fm
import seaborn as sns
import mapclassify
import json
import requests
import datetime as dt
import pandas as pd
import numpy as np
import geopandas as gpd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import plotly_express as px
import matplotlib as mpl
import matplotlib.font_manager as fm
import warnings
import os
import re
from random import *
from geopandas import GeoDataFrame
from sklearn.preprocessing import MaxAbsScaler,RobustScaler
import statsmodels.formula.api as sm 
import random
import seaborn as sns
import matplotlib.pyplot as plt
from haversine import haversine
from sklearn.linear_model import LinearRegression # 선형회귀모델 생성 
from sklearn.model_selection import train_test_split # train/test set 생성 
from sklearn.metrics import mean_squared_error # MSE : 평균제곱오차 - model 평가 

warnings.filterwarnings(action='ignore')

In [ ]:
Region=pd.read_csv('Region.csv')
Gender=pd.read_csv('TimeGender.csv')
covid_case=pd.read_csv('Case.csv')
Patient=pd.read_csv('PatientInfo.csv')

Region.head()

일별 성별별 누적 확진자 및 사망자 추이
- 여성 확진자가 남성 확진자보다 더 많으나 사망자는 남자가 더 많다.

In [ ]:
# date 열을 데이터 타임으로 변환 후, 인덱스로 설정
Gender.date=pd.to_datetime(Gender.date)
Gender=Gender.set_index('date')

male=Gender.loc[Gender['sex']]=='male',:]
female=Gender.loc[Gender['sex']]=='female',:]

In [ ]:
#그래프 한글 환경 설정
%config InlineBackend.figure_format='retina'

#!apt-qq-y install fonts-nanum

import matplotlib.font_manager as fm
fontpath='/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font=fm.FontProperties(fname=fontpath,size=9)
plt.rc('font',family='NanumBarunGothic')
mpl.font_manager._rebuild()

#그래프에 쓰일 폰트 및 사이즈 지정
#font_location= 'C:/Users/ajou/Documents/NanumGothic.ttf'
#fprop=fm.FontProperties(fname=font_location)

#plt.rcParams['font.family'] = 'NanumGothic'
#plt.rcParams['font.size'] = 16
#plt.rcParams['figure.figsize'] = (20,10)

- 일별 누적 확진자 그래프

In [ ]:
#그래픽 크기 설정
plt.rcParams['font.size']=12
plt.rcParams["figure.figsize"]=(14,8)

plt.plot(male['confirmed'],'b',label='male')
plt.plot(female['confirmed'],'r',label='female')
plt.grid()
plt.legend()
plt.title('일별 누적 확진자 추이',fontproperties=font)
plt.xlabel('Month')
plt.ylabel('누적 환자 수',fontproperties=font)
plt.show()

plt.plot(male['deceased'],'b',label='male')
plt.plot(female['deceased'],'r',label='female')
plt.grid()
plt.legend()
plt.title('일별 누적 사망자 추이',fontproperties=font)
plt.xlabel('Month')
plt.ylabel('누적 환자 수',fontproperties=font)
plt.show()

- 지역별 확진자 발생현황 그래프

In [ ]:
#그래픽 크기 설정
plt.reParams["figure.figsize"]=(18,10)

sum_tmp=covid_case.filter(['province','confirmed'])
province_sum=sum_tmp.groupby(['province']).sum()

#지역별 확진자 발생현황 Bar그래프 생성
province_sum['confirmed'].plot.bar(color='#AD8ED8',rot=8,width=0.5)
plt.grid()
plt.legend()
plt.title("지역별 확진자 발생현황",fontproperties=font)
plt.xlabel("지역",fontproperties=font)
plt.ylabel("확진자수")
xpos=np.arrange(len(province_sum.index))
plt.xticks(xpos,list(province_sum.index))

for x,y in enumerate(list(province_sum['confirmed'])):
  num="%d명"%y
  plt.text(x,y,numfontsize=11,color='#5D5D5D',horizontalalignment='center',verticalalignment='bottom')

plt.show()

- 지역별 집단 감영 발생현황 그래프

In [ ]:
#그래픽 크기 설정
plt.reParams["figure.figsize"]=(12,5)

#집단 감염이 많이 일어난 지역을 기준으로 데이터 분할
covid_group_tmp=covid_case[covid_case.group==True]
covid_group=covid_group_tmp[covid_group_tmp.confirmed>=30]
covid_group

#집단 감염이 많이 일어난 지역 파악을 위한 그래프 출력 (한번에 30명 이상의 집단 감염이 있었던 사례)
group_sum_tmp=covid_group.filter(['province','confirmed'])
group_province_sum=group_sum_tmp.groupby(['province']).sum()

group_province_sum['confirmed'].plot.bar(color='#AD8EDB',rot=0,width=0.5)
plt.grid()
plt.legend()
plt.title("지역별 집단 감염 발생현황")
plt.xlabel("지역")
plt.ylabel("확진자 수")
xpos=np.arrange(len(group_province_sum.index))
plt.xticks(xpos,list(group_province_sum.index))

for x,y in enumerate(list(group_province_sum['confirm'])):
  num="%d명"%y
  plt.text(x,y,num,fontsize=11,color='#5D5D5D'),horizontalalignment='center',verticalalignment='bottom')

plt.show()

- 집단 감염 사례 2위 ~6위 그래프
 - 집단 감염 사례의 압도적인 비중을 신천지 사태가 차지하므로 이를 제외한 결과의 분포를 확인한다.

In [ ]:
#그래픽 크기 설정
plt.rcParams["figure.figsize"]=(15,7)

#집단 감염 사례 상위 6개 확인
groupcase=covid_group_tmp.filter(['infection_case','confirmed'])
groupcase_tmp=groupcase.groupby(['infection_case']).sum()

groupcase_top6=groupcase_tmp.sort_values('confirmed',ascending=False).head(6)

#신천지 사례는 이미 독보적으로 그를 제외한 다섯 가지 사례에 대한 pie chart 생성
groupcase_top=groupcase_top6.iloc[1:7,:]

c_map=plt.get_cmap('Spectral')
col=[c_map(i) for i in np.linspace(0,1,5)]

plt.title("집단 감염 사례 2위~6위")
plt.pie(groupcase_top.confirmed,labels=list(groupcase_top.index),autopct='%1.1f%%', shadow=True, colors=col)
plt.show()

In [ ]:
#시,도 단위로 데이터 분할 저장

state_data=covid_case.filter(['province','confirmed'])
state_data=state_data.groupby(['province'],as_index=False).sum()
state_data


In [ ]:
#Time 데이터 읽은 뒤 달에 해당하는 column 추가
#date와 time을 합쳐서 date에 다시 저장한 후 datatime 형식으로 변환

covid_time=pd.read_csv('Time.csv')
covid_time['month']=None
for i in range(len(covid_time)):
  covid_time['month'][i]=int(covid_time['date'][i].split('-')[1])
  
for i in range(len(covid_time)):
  covid_time['date'][i] = "{} {}:00:00".format(covid_time['date'][i],covid_time['time'][i])

covid_time['date']=covid_time['date'].astype('datetime64[ns]')
covid_time

In [ ]:
#확진자 신규 확진자와 신규 완치자 column 추가
covid_time['newconfirmed']=None
covid_time['newconfirmed'][0]=covid_time['confirmed'][0]

covid_time['newreleased']=None
covid_time['newreleased'][0]=covid_time['released'][0]

for i in range(1,len(covid_time)):
  covid_time['newconfirmed'][i]=covid_time['confirmed'][i]=covid_time['confirmed'][i-1]
  covid_time['newreleased'][i] = covid_time['released'][i]-covid_time['released'][i-1]

- 시간에 따른 신규 확진자-완치자 추이(실질확진자) 그래프

In [ ]:
covid_time1=covid_time.filter(['date','newconfirmed','newreleased'])
covid_time1.set_index('date',inplace=True)

covid_time1['newconfirmed'].plot()
covid_time1['newreleased'].plot()
plt.grid()
plt.legend()
plt.title("시간에 따른 신규 확진자-완치자 추이(실질확진자)", fontproperties=font)
plt.xlabel("날짜")
plt.ylabel("신규 확진자 및 신규 완치자 수")

- 위 그래프는 일별 신규 확진자와 신규 완치자 추이를 나타내는 그래프이다. 신규 확진자 수가 신규 완치자 수를 훌쩍 넘는 4월 초중순 정도까지의 구간은 코로나 확진세가 잘 잡히지 않던 정황을 보여주고, 그 이후는 코로나 확진세가 좀 잠잠해졌음을 보여준다.

- 정부의 발빠른 정책으로 코로나가 7월까지는 많이 완화되었음을 알 수 있다. 하지만 분석을 진행중인 8월 말~9월 초 시기에는 아쉽게도 다시 코로나가 유행하고있다.

- 그 중 가장 논란의 중심에 있는 사랑제일교회 광화문 시위의 문제점에 주목할 필요가 있다. 많은 사람들이 모여 시위를 하는 것은 매우 위험하다. 집회 참가자 중 대다수는 집회 이후 귀가 시 감염의 위험을 안은 채 대중교통을 이용하였을 것이다.

- 이로 미루어 볼 때, 대중교통을 통한 코로나 전파 위험을 최소화하는 것을 통해 코로나의 확산 위험을 줄이는 데 일조할 수 있을 것이다. 따라서 대중교토의 혼잡도를 사전에 확인하고 상대적으로 혼잡하지 않은 수단을 이용할 수 있도록 유도하는 것이 중요하다.

In [ ]:
Patient

Age=Patient['age'].value_counts()
AgeDF=pd.DataFrame(Age)

AgeDF_order=AgeDF.sort_index(ascending=True)
AgeDF_order['age']

AgeDF=AgeDF.reset_index()
AgeDF.columns=['age','confirmed']
AgeDF['age']=AgeDF['age'].str[0:2]

AgeDF['age'][8]=0
AgeDF['age'][10]=100

AgeDF['age']=pd.to_numeric(AgeDF['age'])

AgeDF

In [ ]:
AgeDF=AgeDF.sort_values(by=['age'])
AgeDF=AgeDF.set_index('age')
AgeDF

In [ ]:
AgeDF.plot(kind='bar',title="연령별 확진자",rot=0,color='slateblue')
plt.xlabel("연령대")
plt.ylabel("확진자 수 (명)")
plt.rcParams["figure.figsize"] = (10,4)
plt.rcParams['lines.linewidth'] = 2

- 경제활동 인구가 많은 연령대에서 확진자 수가 높은 것을 볼 수 있다. 이는 곧 경제활동이 활발한 연령대들이 상대적으로 위험에 노출되어있음을 의미한다.

In [ ]:
광역시별 데이터와 시군구별 데이터 split
- Region.csv 파일에서 제주도는 시군으로 나눠져 있지 않기 때문에 City Dataframe에도 추가해주었다.
- Region.csv 파일의 오타를 발견해 직접 수정하였다.

In [ ]:
Metropolitan= [dict(Region.loc[index,:])] for index in Region.index if(Region.loc[index,'province']==Region.loc[index,'city'])]
Metropolitan = pd.DataFrame(Metropolitan)

#마지막행 제거
Metropolitan = Metropolitan,iloc[:,-1]
City=[dict(Region.loc[index,:])for index in Region.index if(Region.loc[index,'province']!=Region.loc[index,'city'])]
City=pd.DataFrame(City)
#제주도 추가
City=City.append(Metropolitan.iloc[-1,:],ignore_index=True)
City.tail()

자가격리자 수 분포 확인

In [ ]:
plt.rcParams['font.size']=16
import seaborn as sns
nursing=City['nursing_home_count']
sns.kdeplot(nursing)
plt.title("자가격리자 수 분포")
plt.show()

자가격리자를 정규분포로 만들기 위해 로그 변환
- 이후 회귀 모델 제작에 사용된다.

In [ ]:
nursing=np.log1p(nursing)
City['nursing_home_count']=nursing
sns.kdeplot(nursing)
plt.title("자가격리자 수 분포")
plt.show()

변수간 상관분석 실시

In [ ]:
#위도, 경도, 코드 column 삭제
City_cor=City.drop(['code','province','city','latitude','logitude'],axis=1)
plt.figure(figsize=(8,8))
sns.heatmap(data=City_cor.corr(),annot=True, fmt='.2f',linewidths=5, cmap=sns.diverging_palette(220,20,as_cmap=True)

광역시별로 자가격리자 시각화
- 원의 크기가 클수록 자가격리자 수가 많음을 의미

In [ ]:
#지도 초기화
Covid_map=folium.Map(location=[36,127],tiles="OpenStreetMap",zoom_start=7)
for index in Metropolitan.index:
  lat=Metropolitan.loc[index,'latitude']
  long=Metropolitan.loc[index,'longitude']
  folium.CircleMarker([lat,long], radius = Metropolitan.loc[index,'nursing_home_count']/700,
                     popup = Metropolitan.loc[index,'province'],
                     color = 'red',
                     fill = True).add_to(Covid_map)
Covid_map

시군구별로 자가격리자 데이터 시각화

In [ ]:
Covid_map2 = folium.Map(location=[36, 127], tiles="OpenStreetMap", zoom_start=7)
for index in City.index :
  lat = City.loc[index,'latitude']
  long = City.loc[index,'longitude']
  folium.CircleMarker([lat,long],
                     radius = City.loc[index,'nursing_home_count']/9,
                     popup = City.loc[index,'province'],
                     color = 'blue',
                     fill = True).add_to(Covid_map2)
Covid_map2

shape file 불러오기 및 좌표계 변환
- 시군구별로 자가격리자의 수를 나타내기 위해 시군구별 구역이 나눠져있는 외부데이터 shapefile을 이용하였다.

In [ ]:
gdf=gpd.read_file("SIG.shp",encoding='EUC-KR')
gdf=gdf.to_crs({'init':'epsg:4326'})
gdf.plot()

외부데이터와 내부 데이터 Merge하기
- 데이터를 병합하기 위해서는 기준이 되는 열의 규칙이 같아야 하므로 엑셀파일로 직접 추가적인 작업(Column추가, 지역이름 동일규칙적용)을 실시하였다.
- Merge의 기준이 되는 열은 'province'와 'city'로 설정하였다.

In [ ]:
gdf1=pd.read_excel('Revised_gps_data.xlsx')
gdf1.drop(['Unnamed:0'],axis=1,inplace=True)
gdf.SIG_CD=gdf1['province']
gdf.SIG_KOR_NM=gdf1['city']
gdf=gdf.drop(['SIG_ENG_NM'],axis=1)
gdf.columns=['province','city','geometry']
City=pd.read_excel("Revised_City.xlsx")
Merge_data=pd.merge(left=gdf,right=City,how='left',on=['province','city'],sort=False)
Merge_data.drop(['Column1','code'],axis=1,inplace=True)
Merge_data.head(1)

시군구별 자가격리자 수 데이터 시각화

In [ ]:
fig,ax=plt.subplots(figsize=(10,8))
Merge_data.plot(ax=ax,column="nursing_home_count",cmap="Reds",edgecolor="grey",linewidth=0.4,legend=True)
ax.axis("off")
plt.axis('equal')
plt.show()

- 위 지도를 보면 수도권에 자가격리자가 집중되어 있는 것을 확인할 수 있다. 이 보고서에는 전국민의 약 1/5이 거주하고 있는 서울을 중심으로 분석하고자 한다. 

#### 2. 교통 데이터 전처리

#### 3. 대중교통 안전경로 추천 프로세스 제작
1) 데이터 활용
- 지하철 역별 이용 인원 데이터 (출처 : 서울열린데이터광장)
- 버스 정류장별 이용 인원 데이터 (출처 : 한국교통안전공단)
 - 교통카드 데이터의 정류장별 이용량 테이블을 활용
- 지하철 역별 위도 경도 좌표 데이터
 - 1번 데이터의 지하철역 목록을 Google SpreadSheet의 확장프로그램인 GeoCode By Awesome Table 기능을 이용하여 매핑
- 버스 정류장 위도 경도 좌표 데이터 (출처 : 서울열린데이터광장)
 - 버스 정류장별 위치 정보 데이터 활용
- 확진자 방문장소 데이터
 - 코로나있다 사이트 활용하여 직접 수집
- 경로 추천을 위한 데이터 (출처 : ODsay API)
 - 출발지와 도착지 입력에 따른 길찾기 API 활용

 2) 안전점수 계산을 위한 지표 선정
 - 경로상의 출발지, 환승지, 도착지에 해당하는 지하철,버스 역의 혼잡도
 - 경로상의 환승지 반경 3km내 확진자 방문장소의 수

필요 데이터 불러오기

In [ ]:
# 지하철역 이용량 데이터
Subway = pd.read_csv('Subway_Popularity_July.csv', encoding='cp949')

# 버스정류장 이용량 데이터
Bus = pd.read_csv('BUS3_2.csv', encoding='cp949')

# 지하철역 위도 경도 데이터
SubwayLongLat = pd.read_csv('SubwayLocation.csv', encoding='utf-8')

# 버스정류장 위도 경도 데이터
BusLongLat = pd.read_csv('BusLocation.csv', encoding='cp949')

# 확진자 방문장소 데이터
Visitor = pd.read_csv('Place_Visitors.csv', encoding='utf-8')

데이터 전처리
Subway 데이터
- 사용하고자 하는 column은 요일, 역명, 시간 변수들
- 필요없는 column은 삭제하고 시간변수 column명들을 쓰기 용이하게 변환
- 요일과 역명에 따른 시간대별 합계 정보가 필요하기 때문에, 요일과 역명으로 그룹화하여 합계를 낸 데이터를 Subway_DayOfWeek 변수에 저장

In [ ]:
ColumnList=[]
NameList=[]

#column명 전처리
Subway.columns=Subway.columns.str.replace(" ","")
Subway.columns.values[:11]=Subway.columns[:11].str.replace("0","")
for Column in Subway.columns:
  ColumnList.append(Column.split('~')[0])
Subway.columns=ColumnList

#column명 변경 및 필요없는 column 삭제
Subway['역명'].str.split('(')[0]
Subway = Subway.rename({'6시이전':'5','24시이후':'24'}, axis='columns')
Subway = Subway.drop(['구분','할인','호선','역번호','합계'], axis='columns')

#필요한 열 기준으로 그룹화하여 합산
Subway=Subway.groupby(['날짜','역명']).sum()
Subway=Subway.reset_index()

#역명 뒤에 () 붙으면 그 앞까지만 저장
for Station in Subway['역명']:
  NameList.append(Station.split('(')[0])
Subway['역명']=NameList

#날짜형식 변환 및 요일 column추가
Subway['날짜']=pd.to_datetime(Subway['날짜'])
Subway.insert(1,'요일',Subway['날짜'].dt.dayofweek)

#필요한 열 기준으로 그룹화하여 합산
Subway_DayOfWeek=Subway.groupby(['요일','익명']).sum()
Subway_DayOfWeek=Subway_DayOfWeek.reset_index()

Subway_DayOfWeek.head()


지하철 역별 위도 경도 좌표 데이터
- Subway_DayOfWeek 데이터의 역명 형식에 맞게 역명을 변환해주고 column명을 변환해줌

In [ ]:
#불필요한 열 제거
SubwayLongLat=SubwayLongLat.drop(['Unnamed:3','place'],axis='columns')

#Subway_DayOfWeek 데이터 프레임에 맞게 문자열 처리, column명 변환
SubwayLongLat['address']=SubwayLongLat['address'].str[:-1]
SubwayLongLat['address'][SubwayLongLat['address']=='서울']='서울역'
SubwayLongLat=SubwayLongLat.rename({'address':'역명'},axis='columns')
SubwayLongLat.head()

버스 정류장별 이용 인원 데이터
- 시간, 요일, 역명에 따른 승차인원의 합계정보.
- 시간, 요일, 역명을 기준으로 그룹화하여 합계 낸 데이터 Bus_DayOfWeek에 저장.

In [ ]:
#날짜 형식으로 변환
Bus['DATE']=Bus['DATE'].astype(str)
Bus['DATE']=Bus['DATE'].str[:4]+'-'+Bus['DATE'].str[4:6]+'-'+Bus['DATE'].str[-2:]
Bus['DATE']=pd.to_datetime(Bus['DATE'])

#필요없는 column제거 및 요일 column추가
Bus.columns= ['시간', '날짜', '시군구코드','역번호','역명','승차인원','하차인원']
Bus.insert(1,'요일',Bus['날짜'].dt.dayofweek)
Bus=Bus.drop(['날짜','시군구코드','역변호','하차인원'],axis='columns')

#필요한 열 기준으로 그룹화하여 합산
Bus_DayOfWeek=Bus.groupby(['시간','요일','역명']).sum()
Bus_DayOfWeek=Bus_DayOfWeek.reset_index()
Bus_DayOfWeek.head()

버스 정류장별 위도 경도 좌표 데이터
- 필요한 col만 남기고 이용하기 용이하게 col명 변경

In [ ]:
BusLongLat = BusLongLat.drop(['표준ID','ARS-ID','비고'], axis='columns')
BusLongLat = BusLongLat.rename({'정류장명':'역명', 'X좌표':'Longitude', 'Y좌표':'Latitude'}, axis='columns')
BusLongLat.head()

### 함수 구축
1. TimeRoute 함수
- api를 통해 받아오게 될 추천경로 Json 파일을 참조하기 좋게, 그리고 보기 쉽게 정리하고자 하는 함수이다.
- input값으로 받은 json 파일에서 경로 유형, 총 소요시간, 교통수단 유형,호선 및 노선, 역 이름 등의 정보를 추출하여 하나의 리스트로 결괏값을 반환한다.

In [ ]:
def TimeRoute(JsonFile):
  RouteList=[]

  #경로마다 PathType,TotalTime 저장
  for PathType in JsonFile['result']['path']:
    Route=[{'PathType':PathType['pathType'],'TotalTime':PathType['info']['totalTime']}]

    #trafficType 확인 후 해당 데이터셋에서 조회하여 필요한 값들 저장
      for Path in PathType['subPath'][1:-1]:
      if Path['trafficType'] == 1:
        Route.append({'TrafficType' : Path['trafficType'], 'StationID' : Path['startID'], 'Lane' : Path['lane'][0]['name'], 'StationName' : Path['startName']})

      elif Path['trafficType'] == 2:
        Route.append({'TrafficType' : Path['trafficType'], 'StationID' : Path['startID'], 'Lane' : Path['lane'][0]['busNo'], 'StationName' : Path['startName']})

      Route.append({'StationID':Path['endID'],'StationName' : Path['endName']})
      RouteList.append(Route)

return(RouteList)

2. NumOfPeople 함수
- 1번 함수를 통해 정리된 자료를 인풋 값으로 받아 각 정류장 및 역이 현재 요일 및 시간대에 어느정도의 사람이 이용했는지 정리하는 함수
- 각 루트의 경로 유형과 교통수단 유형을 참조하여 버스인지 지하철인지 파악 후 해당 데이터프레임에서 이용인원을 조회하여 리스트에 추가하고 이를 결과값으로 반환한다

In [ ]:
def NumOfPeople(TimeRouteFile):
  DayOfWeek = dt.datetime.now().weekday()
  RouteTime = dt.datetime.now().hour
  RouteSumList = []
  TypeList = []

  # PathType과 TrafficType을 확인하여 적절한 데이터셋으로부터 유동인구를 저장
   for Route in TimeRouteFile:
    RouteSum = []
    Type = []
    if Route[0]['PathType'] == 1:
      for Station in Route[1:]:
        RouteSum.append(sum(Subway_DayOfWeek['{}'.format(RouteTime)][Subway_DayOfWeek['요일'] == DayOfWeek][Subway_DayOfWeek['역명'] == Station['StationName']]))
        Type.append(1)

    elif Route[0]['PathType'] == 2:
      for Station in Route[1:]:
        RouteSum.append(sum(Bus_DayOfWeek['승차인원'][Bus_DayOfWeek['시간']==RouteTime][Bus_DayOfWeek['요일']==DayOfWeek][Bus_DayOfWeek['역명']==Station['StationName']]))
        Type.append(2)
    
    else:
      for Station in Route[1:-1]:
        if Station['TrafficType'] == 1:
          RouteSum.append(sum(Subway_DayOfWeek['{}'.format(RouteTime)][Subway_DayOfWeek['요일'] == DayOfWeek][Subway_DayOfWeek['역명'] == Station['StationName']]))
          Type.append(1)
        else:
          RouteSum.append(sum(Bus_DayOfWeek['승차인원'][Bus_DayOfWeek['시간']==RouteTime][Bus_DayOfWeek['요일']==DayOfWeek][Bus_DayOfWeek['역명'] == Station['StationName']]))
          Type.append(2)
      if Station['TrafficType'] == 1:
        RouteSum.append(sum(Subway_DayOfWeek['{}'.format(RouteTime)][Subway_DayOfWeek['요일'] == DayOfWeek][Subway_DayOfWeek['역명'] == str(Route[-1]['StationName'])]))
        Type.append(1)
      else:
        RouteSum.append(sum(Bus_DayOfWeek['승차인원'][Bus_DayOfWeek['시간']==RouteTime][Bus_DayOfWeek['요일']==DayOfWeek][Bus_DayOfWeek['역명']==Route[-1]['StationName']]))
        Type.append(2)
      
    TypeList.append(Type)
    RouteSumList.append(RouteSum)



  return(RouteSumList,TypeList)

3. RouteIndex 함수
- 길찾기 API를 통해 많은 수의 경로가 추천되는데, 총 소요시간 기준 상위 5개 정도의 경로가 아니면 소요시간이 매우 길어 의미가 없는 경로라고 판단하였다
- 또한, 경유하는 정류장들은 같지만 버스 번호만 달라 정류장별 혼잡도는 똑같은 경로들이 존재한다.
- 이 함수는 경유하는 정류장과 역들이 완전히 일치하는 경로들은 하나만 남기면서 상위 5개 경로들의 정보와 인덱스를 남기는 함수이다.

In [ ]:
def RouteIndex(NumOfPeopleFile):
  NewNumList = []
  IndexList = []
  NewIndexList = []

  # 중복 경로 제거하는 반복문 생성
  for idx, NumList in enumerate(NumOfPeopleFile[0]):
    if NumList not in NewNumList:
      NewNumList.append(NumList)
      IndexList.append(idx)
      NewIndexList.append(NumOfPeopleFile[1][idx])

  NewNumList = NewNumList[:5]
  IndexList = IndexList[:5]
  NewIndexList = NewIndexList[:5]

  return(NewNumList, IndexList, NewIndexList)

4. CongestionLevel 함수
- 혼잡도 점수 계산 프로세스 <지표 1>

1) 지하철 역과 버스 정류장의 유동인구 수준의 차이가 존재하므로 지하철 역의 기초 level 단위를 1000으로, 버스 정류장의 기초 level 단위를 115로 지정한다.
  
2) 기초 level 단위가 임의로 정해진 만큼, 버스 level 척도와 지하철 level 척도의 큰 차이가 발생할 수 있다. 이를 방지하기 위해서는 버스는 최대로 부여될 수 있는 level이 어느 정도이고 지하철은 어느정도인지 알아내어 그 정도가 비슷해질 수 있도록 Scaling하는 작업이 필요하다.

3) 먼저, 해당 요일, 시간대의 지하철역과 버스 정류장 유동인구 최대값을 기초 level로 나누어 최대로 지정될 수 있는 level이 어느 정도인지 계산한다.

4) 만약 버스 최대 level이 50이고, 지하철 최대 level이 100이라면, (버스 최대 level/지하철 최대 level), 즉, 0.5를 지하철 최종 지표에 곱하여 양 지표 척도를 비슷하게 만들 수 있다. 따라서, 각각의 최대 level 수준을 서로 나누어 최종 점수 Scaling에 이용할 비율 지표 'Ratio'를 구한다.

5) 3번 함수에서 구한 5개의 경로 내 역들의 유동 인구 수를 기초 level 단위로 나누어 유동 인구 level을 구하고, 지하철역 level에 미리 구한 Ratio를 곱하여 버스 level 척도와 비슷한 수준으로 스케일링을 해준다.

6) 구해진 점수들을 각 경로에 대한 상대 점수로 환산한다.

In [ ]:
def CongestionLevel(RouteIndexFile):
  DayOfWeek = dt.datetime.now().weekday()
  RouteTime = dt.datetime.now().hour

  # 기초 level 단위 지정
  PopSubLevel = 1000
  PopBusLevel = 115

  # 최대 가능 level 계산
  MaxPopSub = np.max(Subway_DayOfWeek['{}'.format(RouteTime)][Subway_DayOfWeek['요일'] == DayOfWeek])//PopSubLevel + 1
  MaxPopBus = np.max(Bus['승차인원'][Bus['시간'] == RouteTime][Bus['요일'] == DayOfWeek])//PopBusLevel + 1
  
  # Scaling Ratio 계산
  Ratio = MaxPopBus / MaxPopSub

  LevelList1 = []
  LevelList = []

  # 혼잡도 점수 계산
  for i in range(len(RouteIndexFile[1])):
    Level = []
    for j in range(len(RouteIndexFile[2][i])):
      if RouteIndexFile[2][i][j] == 1:
        Level.append((RouteIndexFile[0][i][j] // PopSubLevel + 1) * Ratio)
      
      else:
        Level.append(RouteIndexFile[0][i][j] // MaxPopBus + 1)
    
    LevelList1.append(round(sum(Level), 4))

  for Levels in LevelList1:
    LevelList.append(round(Levels/sum(LevelList1),4))
  
  return(LevelList)

5. TransferStation 함수

- 두 번째 지표인 반경 3km 내 확진자 방문 장소 수를 계산할 때, 출발지와 도착지는 그 위치가 큰 차이가 있기 힘들기 때문에, 환승 지역만을 기준으로 계산해야 한다.
- 따라서, 이 함수를 통해 경로들의 환승역만 모아 결과값으로 반환한다.

In [ ]:
def TransferStation(TimeRouteFile):
  Idx = RouteIndex(NumOfPeople(TimeRouteFile))[1]
  TransferList = []

  # 환승지 데이터만 빼서 저장
  for idx in Idx:
    TransferStation = []
    for Station in TimeRouteFile[idx][1:-1]:
      TransferStation.append({'TrafficType' : Station['TrafficType'], 'StationName' : Station['StationName']})
    TransferList.append(TransferStation)
  
  return(TransferList)

6. LongLat 함수
- 확진자 방문장소가 반경 내에 있는지 확인하기 위해서는 환승 지역의 위치 좌표도 필요하다.
- 따라서 이 함수에서는 5번 함수에서 도출한 경로 내 환승 지역들의 위도 경도 좌표를 데이터프레임으로부터 찾아 딕셔너리 형태로 리스트에 저장하여 반환한다.

In [ ]:
def LongLat(TransferStationFile):
  LongLatList = []
  for Stations in TransferStationFile:
    LongLat = []

    # TrafficType 확인 후 적절한 데이터 셋에서 위도와 경도 좌표 조회하여 저장
    for Station in Stations:
      if Station['TrafficType'] == 1:
        LongLat.append({'Latitude' : np.max(SubwayLongLat['Latitude'][SubwayLongLat['역명']==Station['StationName']].astype('float')), 'Longitude' : np.max(SubwayLongLat['Longitude'][SubwayLongLat['역명']==Station['StationName']].astype('float'))})
      else:
        LongLat.append({'Latitude' : np.max(BusLongLat['Latitude'][BusLongLat['역명']==Station['StationName']].astype('float')), 'Longitude' : np.max(BusLongLat['Longitude'][BusLongLat['역명']==Station['StationName']].astype('float'))})
    
    LongLatList.append(LongLat)
  
  return(LongLatList)

7. DangerLevel 함수
반경 내 확진자 방문장소 수 프로세스 <지표 2>

In [ ]:
def DangerLevel(LongLatFile):
  CountList = []
  CountList1 = []
  for Long_Lats in LongLatFile:
    CountList2 = []
    for Long_Lat in Long_Lats:
      Count = 0

      # haversine 모듈 이용하여 거리 계산 및 count
      for i in range(len(Visitor)):
        if haversine((Long_Lat['Latitude'], Long_Lat['Longitude']), (Visitor['Latitude'][i],Visitor['Longitude'][i]), unit = 'km') < 3:
          Count += 1
      CountList2.append(Count)
    CountList1.append(sum(CountList2))
  
  for Counts in CountList1:
    CountList.append(round(Counts/sum(CountList1)*0.4,4))


  return(CountList)

8. Recommendation 함수

In [ ]:
def Recommendation(TimeRouteFile, RouteIndexFile, CongestionLevelFile, DangerLevelFile):
  TotalPoint = []
  RouteList = []

  # 위험도 지표 2개 합산하여 총점 산출
  for i in range(len(CongestionLevelFile)):
    TotalPoint.append(CongestionLevelFile[i] + DangerLevelFile[i])
  
  # 총점 중 최소값 찾기 (가장 안전한 경로 찾기)
  for idx, point in enumerate(TotalPoint):
    if point == min(TotalPoint):
      Index = idx
  
  # 경로 저장
  for i in range(len(RouteIndexFile[1])):
    Route = []
    for StationInfo in TimeRouteFile[i][1:-1]:
      Route.append('({}){}'.format(StationInfo['Lane'], StationInfo['StationName']))
    Route.append('{}'.format(TimeRouteFile[i][-1]['StationName']))
    RouteList.append(Route)

  BestRoute = RouteList[Index]

  return(RouteList,BestRoute,Index)

API를 통해 받아온 경로 Json 파일 불러오기

In [ ]:
with open('transport.json', encoding='UTF-8-sig') as json_file:
  TransportJson = json.load(json_file)

In [ ]:
a = TimeRoute(TransportJson)
b = NumOfPeople(a)
c = RouteIndex(b)
d = CongestionLevel(c)

e = TransferStation(a)
f = LongLat(e)
g = DangerLevel(f)

h = Recommendation(a,c,d,g)

Route = h[0]
BestRoute = h[1]
BestIndex = h[2]

print("""목적지까지의 경로입니다.

경로 1 : {} / {}분 소요
경로 2 : {} / {}분 소요
경로 3 : {} / {}분 소요
경로 4 : {} / {}분 소요
경로 5 : {} / {}분 소요


안전 지표 기반 최적 경로 :
***********************************************************************
{} / {}분 소요
***********************************************************************


(위 결과는 경로 상의 출발지, 환승지, 도착지의 혼잡도 지표와 환승지 근처의 확진자 방문장소 수 지표를 기반으로 추천된 경로입니다.)
""".format(' -> '.join(Route[0]), a[c[1][0]][0]['TotalTime'],
           ' -> '.join(Route[1]), a[c[1][1]][0]['TotalTime'],
           ' -> '.join(Route[2]), a[c[1][2]][0]['TotalTime'],
           ' -> '.join(Route[3]), a[c[1][3]][0]['TotalTime'],
           ' -> '.join(Route[4]), a[c[1][4]][0]['TotalTime'],
           ' -> '.join(BestRoute), a[c[1][BestIndex]][0]['TotalTime']))

### 신(新)코로나 지표 제작

특정 지역의 코로나 위험도를 판단할 때 단순 확진자 뿐만 아니라 지역적 특성을 나타내는 각종 지표들을 이용해 특정 지역의 현재 위험도를 표현할 수 있는 보완된 신(新)코로나 지표가 필요하다.

#### 모델링 설계 과정
1. 모델의 현재 지표로 판단한 위험도가 몇일의 간격(적절한 lag 변수 값)을 두고 나타나는지 파악한다.
- 다중선형회귀 분석은 독립변수나 종속변수 중 최소한 하나는 정규분포의 형태를 띄어야 하기 때문에 코로나 확진자 지표의 자연로그 함수를 사용하여 정규분포 형태로 변환하였다. 
- 한국 기준 코로나 증상발현시간은 약 4일이므로 lag 변수를 5일, 7일, 10일로 설정해서 각각 데이터 셋을 만든다.
- 6월 10일을 기준으로 6월 15일, 6월 17일, 6월 20일을 만든다.
- 차례대로 다중회귀분석을 실시하여, 가장 적절한 lag변수를 찾는다.

In [ ]:
data = pd.read_csv("regression123.csv")
data = data[['city','market', 'res', 'ele_cnt', 'kind_cnt','uni_cnt', 'eld_p_r', 'eld_a_r','aca_r','6_10','6_15']]
data.head()

In [ ]:
sns.kdeplot(data['6_15'])
plt.title("확진자 수 분포")
plt.show()

In [ ]:
sns.kdeplot(np.log1p(data['6_15'])) #로그변환
plt.title("확진자 수 분포")
plt.show()

In [ ]:
#스케일링 함수 정의
scaler = MaxAbsScaler()
#확진자 수 로그 변환
data['6_10'] = np.log1p(data['6_10'])
data['6_15'] = np.log1p(data['6_15'])
data = data.drop(['city'],axis=1)
data[data.columns.values[:]] = scaler.fit_transform(data[data.columns.values[:]])
plt.figure(figsize=(8,8))
sns.heatmap(data = data.corr(), annot=True, 
fmt = '.2f', linewidths=.5, cmap=sns.diverging_palette(220,20,as_cmap=True))

lag 변수: 5일 설정했을 때 오류율 구하기

sklearn 라이브러리 사용한 다중선형회귀 모델 구축
- 모델의 학습 및 검증을 위해 7:3 랜덤샘플링
- 오류 지표로 MSE 사용
- 성능 측정을 위해 10번 테스트, 10개의 MSE 평균 구함
- lag 변수를 5로 설정 했을 때 평균 MSE 값은 -로 측정됨

In [ ]:
#열이름 가져오기.
cols = list(data.columns) 
#독립변수 열이름.
x_cols = cols[:-1] 
#종속변수 열이름
y_col = cols[-1] 
#매번 모델 성능을 측정할 리스트 생성
score = []
for i in range(10) :
  #train/test split (70 : 30) 
  train_set, test_set = train_test_split(data, test_size = 0.3) 
  #모델 생성 및 학습
  model = LinearRegression() 
  model.fit(X = train_set[x_cols], y = train_set[y_col]) 
  #모델을 통한 예측
  y_pred = model.predict(X=test_set[x_cols]) 

  #정답 
  Y = test_set[y_col] 
  #MSE산출 및 score 리스트에 추가
  mse = mean_squared_error(y_true = Y, y_pred = y_pred) 
  score.append(mse)
#10번의 MSE의 평균 값 산출
final_score = {'lag : 5' :sum(score) / len(score)}
final_score

lag 변수: 5일 설정했을 때 오류율 구하기 
- 6월 17일 확진자를 종속변수로

In [ ]:
data = pd.read_csv("regression123.csv")
data = data[['city','market', 'res', 'ele_cnt', 'kind_cnt','uni_cnt', 'eld_p_r', 'eld_a_r','aca_r','6_10','6_17']]
#스케일링 함수 정의
scaler = MaxAbsScaler()
#확진자 수 로그 변환
data['6_10'] = np.log1p(data['6_10'])
data['6_17'] = np.log1p(data['6_17'])
data = data.drop(['city'],axis=1)
#열이름 가져오기.
cols = list(data.columns) 
#독립변수 열이름.
x_cols = cols[:-1] 
#종속변수 열이름
y_col = cols[-1] 
#매번 모델 성능을 측정할 리스트 생성
score = []
for i in range(10) :
  
  #train/test split (70 : 30) 
  train_set, test_set = train_test_split(data, test_size = 0.3) 
  
  #모델 생성 및 학습
  model = LinearRegression() 
  model.fit(X = train_set[x_cols], y = train_set[y_col]) 
  
  #모델을 통한 예측
  y_pred = model.predict(X=test_set[x_cols]) 
  
  #정답 
  Y = test_set[y_col] 
  
  #MSE산출 및 score 리스트에 추가
  mse = mean_squared_error(y_true = Y, y_pred = y_pred) 
  score.append(mse)

#10번의 MSE의 평균 값 산출
mean_score = sum(score) / len(score)
final_score['lag : 7'] = mean_score
final_score

lag 변수 : 10일 설정 했을 때 오류율 구하기
- 6월 20일 확진자를 종속변수로

In [ ]:
data = pd.read_csv("regression123.csv")
data = data[['city','market', 'res', 'ele_cnt', 'kind_cnt','uni_cnt', 'eld_p_r', 'eld_a_r','aca_r','6_10','6_20']]
#스케일링 함수 정의
scaler = MaxAbsScaler()
#확진자 수 로그 변환
data['6_10'] = np.log1p(data['6_10'])
data['6_20'] = np.log1p(data['6_20'])
data = data.drop(['city'],axis=1)
#열이름 가져오기.
cols = list(data.columns) 
#독립변수 열이름.
x_cols = cols[:-1] 
#종속변수 열이름
y_col = cols[-1] 
#매번 모델 성능을 측정할 리스트 생성
score = []
for i in range(10) :
  
  #train/test split (70 : 30) 
  train_set, test_set = train_test_split(data, test_size = 0.3) 
  
  #모델 생성 및 학습
  model = LinearRegression() 
  model.fit(X = train_set[x_cols], y = train_set[y_col]) 
  
  #모델을 통한 예측
  y_pred = model.predict(X=test_set[x_cols]) 
  
  #정답 
  Y = test_set[y_col] 
  
  #MSE산출 및 score 리스트에 추가
  mse = mean_squared_error(y_true = Y, y_pred = y_pred) 
  score.append(mse)

#10번의 MSE의 평균 값 산출
mean_score = sum(score) / len(score)
final_score['lag : 10'] = mean_score
final_score

lag=5가 가장 유효하다고 판단됨

2. 모델의 유효한 변수 추출

- lag 변수는 5로 밝혀졌으므로, 6월 15일 데이터를 독립변수, 6월 20일 데이터를 종속변수로 설정하여 모델을 검증한다.
- 다중선형회귀분석에서 독립변수간 다중공선성 문제 해결 및 유효하지 않는 변수를 제거하기 위해 Backward Elimination 방법을 사용해 적절한 변수를 선택한다.
- p-value가 가장 높게 나온 uni_cnt(대학교 수), market(대형점포 수) 변수를 제거한다.

In [ ]:
data = pd.read_csv("regression123.csv")
data = data[['market', 'res', 'ele_cnt', 'kind_cnt','uni_cnt', 'eld_p_r', 'eld_a_r','aca_r','6_15','6_20']]
scaler = MaxAbsScaler()
#변수 이름 재설정
data['patient'] = np.log1p(data['6_15'])
data['lag_patient'] = np.log1p(data['6_20'])
data = data.drop(['6_15','6_20'],axis=1)
data[data.columns.values[:]] = scaler.fit_transform(data[data.columns.values[:]])
result = sm.ols(formula = 'lag_patient ~ market+res+	ele_cnt+	kind_cnt+	uni_cnt+	eld_p_r+	eld_a_r+ aca_r + patient', data = data).fit()
result.summary()

In [ ]:
#p-value가 가장 높게 나온 uni_cnt(대학교 수), market(대형점포 수) 변수를 제거한다.
data = data.drop(['uni_cnt','market'],axis=1)
result = sm.ols(formula = 'lag_patient ~ kind_cnt+eld_p_r+res+	ele_cnt+	eld_a_r+ aca_r + patient', data = data).fit()
result.summary()

In [ ]:
#p-value가 가장 높게 나온 aca_r(학원 비율), eld_p_r(노인비율) 변수를 제거한다.
data = data.drop(['aca_r','eld_p_r'],axis=1)
result = sm.ols(formula = 'lag_patient ~ kind_cnt+res+	ele_cnt+	eld_a_r + patient', data = data).fit()
result.summary()

In [ ]:
#p-value가 가장 높게 나온 kind_cnt(유치원 수), eld_cnt(노인 수) 변수를 제거한다.
data = data.drop(['kind_cnt','ele_cnt'],axis=1)
result = sm.ols(formula = 'lag_patient ~ res +	eld_a_r + patient', data = data).fit()
result.summary()

최종모델의 변수: 식당 수, 독거노인 비율, 현재 확진자 수

3. 지표 검증

In [ ]:
# 1. 확진자 수를 통해 예측한 5일 뒤 확진자 수

data['x'] = data['patient']
data['y'] = data['lag_patient']
data = data.drop(['res','eld_a_r','lag_patient','patient'],axis = 1)
result = sm.ols(formula = 'y ~ x', data = data).fit()
R_squared = {'Now_patient_prediction R_squred' : result.rsquared}
R_squared

# R_squared = 0.9822688115543752

In [ ]:
#2. 모델을 통해 5일 뒤 확진자 수 예측과 R-squared 값 도출

data = pd.read_csv("regression123.csv")
data = data[['city','res','eld_a_r','6_15','6_20']]
scaler = MaxAbsScaler()
#변수 이름 재설정
data['patient'] = np.log1p(data['6_15'])
data['lag_patient'] = np.log1p(data['6_20'])
data = data.drop(['6_15','6_20'],axis=1)
data[data.columns.values[1:]] = scaler.fit_transform(data[data.columns.values[1:]])
result = sm.ols(formula = 'lag_patient ~ res +	eld_a_r + patient', data = data).fit()
R_squared = {'Model_patient_prediction R_squared' : result.rsquared}
R_squared

#R_squared = 0.9835655150159914 -->모델로 예측하는 것이 더 낫다

### 모델로 예측한 순위로 시군구별 위험도 표시하기
- 데이터를 제공하는 가장 마지막 날짜 기준으로 위험도 작성
- 상위 25개 시군구는 매우 위험, 26-199위는 50개씩 split해 위험, 보통, 비교적 안전, 안전으로 재분류

In [ ]:
data = pd.read_csv("regression123.csv")
data = data[['city','res','eld_a_r','6_30']]
data['patient'] = np.log1p(data['6_30'])
data = data.drop(['6_30'],axis=1)
data[data.columns.values[1:]] = scaler.fit_transform(data[data.columns.values[1:]])
#모델을 통한 위험도 값 예측
prediction = result.predict(data.iloc[:,1:])
prediction

In [ ]:
#예측 값 column으로 추가
data['prediction'] = prediction
#분류 column생성
data['classification'] = 0
#예측 값 정렬
data= data.sort_values(by='prediction' ,ascending=False)
#예측 값을 기반으로 classification
for i in range(len(data)):
  #상위 25위 도시 매우 위험
  if (data.iloc[i,4] >= data.iloc[24,4]):
    data.iloc[i,5] = "매우 위험"
  #상위 26~75위 도시 위험
  elif (data.iloc[i,4] >= data.iloc[74,4]):
    data.iloc[i,5] = "위험"
  #상위 76 ~ 125위 도시 보통
  elif (data.iloc[i,4] >= data.iloc[124,4]):
    data.iloc[i,5] = "보통"
  #상위 126 ~ 175위 도시 비교적 안전
  elif (data.iloc[i,4] >= data.iloc[174,4]):
    data.iloc[i,5] = "비교적 안전"
  else : 
  #하위 50개 도시 안전
    data.iloc[i,5] = "안전"

### 결론
-  '현재의 확진자가 과연 몇일 뒤의 확진자를 대변할 수 있는가?' 라는 의학적으로 접근해온 문제를 통계적으로 접근하며 사용자들에게 실질적으로 현 상황의 위험도를 나타낼 수 있도록 진행하였다. 우리가 앞으로 더 나은 선행지표를 만들고자 한다면 코로나19에 많은 영향을 미쳤을 것이라 짐작할 수 있는 변수를 넣어 분석을 진행할 수 있을 것.